In [ ]:
import geopandas as gpd

In [ ]:
# http://www.resdc.cn/data.aspx?DATAID=200
map_cn = gpd.read_file('../map_of_China/sheng/CN-sheng-A.shp')

In [ ]:
map_cn.head()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots()
vmin, vmax = 0, 1e+12
map_cn.plot(column='AREA', cmap='Reds', linewidth=0.5, edgecolor='black', figsize=(10,10), legend=True, ax=ax, norm=plt.Normalize(vmin=vmin, vmax=vmax), vmin=vmin, vmax=vmax)

In [ ]:
map_cn.query('not name.isnull()') # name 缺少"香港"/"澳门"

In [ ]:
sheng_map = dict()
for row in map_cn.iterrows():
    sheng_name = row[1]['name']
    if type(sheng_name) is str:  # 有省名称的数据行
        sheng_map[sheng_name] = row[1]
        
print('\n'.join(sheng_map.keys()))
print(len(sheng_map))

In [ ]:
sheng_id = dict()
for name in sheng_map:
    id = sheng_map[name]['SHENG']  # 整型
    sheng_id[id] = name  # 可对应到省名的'SHENG'

no_named_ad = set()
for row in map_cn.iterrows():
    id = row[1]['SHENG']
    if id not in sheng_id:
        no_named_ad.add(id)  # 不可对应到省名的'SHENG', 即为"香港"/"澳门"

assert(len(no_named_ad) == 2)
assert(81 in no_named_ad)  # 'SHENG' == 81 代表香港
assert(82 in no_named_ad)  # 'SHENG' == 82 代表澳门
print(*list(no_named_ad))

In [ ]:
xianggang_map = map_cn.loc[map_cn['SHENG']==81, :]
aomen_map = map_cn.loc[map_cn['SHENG']==82, :]
guangdong_map = map_cn.loc[map_cn['SHENG']==44, :]

In [ ]:
fig, ax = plt.subplots()
vmin, vmax = 0, 1e+12
xianggang_map.plot(column='AREA', cmap='Reds', linewidth=0.5, edgecolor='black', figsize=(10,10), legend=True, ax=ax, norm=plt.Normalize(vmin=vmin, vmax=vmax), vmin=vmin, vmax=vmax)

In [ ]:
fig, ax = plt.subplots()
vmin, vmax = 0, 1e+12
aomen_map.plot(column='AREA', cmap='Reds', linewidth=0.5, edgecolor='black', figsize=(10,10), legend=True, ax=ax, norm=plt.Normalize(vmin=vmin, vmax=vmax), vmin=vmin, vmax=vmax)

In [ ]:
fig, ax = plt.subplots()
vmin, vmax = 0, 1e+12
guangdong_map.plot(column='AREA', cmap='Reds', linewidth=0.5, edgecolor='black', figsize=(10,10), legend=True, ax=ax, norm=plt.Normalize(vmin=vmin, vmax=vmax), vmin=vmin, vmax=vmax)

In [ ]:
import pandas as pd

In [ ]:
# https://github.com/CSSEGISandData/COVID-19
covid_19_confirmed = pd.read_csv('../COVID-19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')

In [ ]:
covid_19_confirmed.head()

In [ ]:
covid_19_confirmed.query('`Country/Region`=="China"')

In [ ]:
covid_19_confirmed.head()

covid_19_confirmed.loc[(covid_19_confirmed['Country/Region'] == 'Taiwan*'),'Province/State']='Taiwan'
covid_19_confirmed.loc[(covid_19_confirmed['Country/Region'] == 'Taiwan*'),'Country/Region']='China'

covid_19_confirmed[covid_19_confirmed['Country/Region'] == 'China']

In [ ]:
sheng_name_to_id = {
    'Beijing': 11,
    'Tianjin': 12,
    'Hebei': 13,
    'Shanxi': 14,
    'Inner Mongolia': 15,
    'Liaoning': 21,
    'Jilin': 22,
    'Heilongjiang': 23,
    'Shanghai': 31,
    'Jiangsu': 32,
    'Zhejiang': 33,
    'Anhui': 34,
    'Fujian': 35,
    'Jiangxi': 36,
    'Shandong': 37,
    'Henan': 41,
    'Hubei': 42,
    'Hunan': 43,
    'Guangdong': 44,
    'Guangxi': 45,
    'Hainan': 46,
    'Chongqing': 50,
    'Sichuan': 51,
    'Guizhou': 52,
    'Yunnan': 53,
    'Tibet': 54,
    'Shaanxi': 61,
    'Gansu': 62,
    'Qinghai': 63,
    'Ningxia': 64,
    'Xinjiang': 65,
    'Taiwan': 71,
    'Hong Kong': 81,
    'Macau': 82
}

In [ ]:
covid_19_confirmed_China = covid_19_confirmed[covid_19_confirmed['Country/Region'] == 'China']

In [ ]:
for row in covid_19_confirmed_China.iterrows():
    name = row[1]['Province/State']
    assert(name in sheng_name_to_id)

In [ ]:
from datetime import datetime
import matplotlib.colors as colors

def plot_normalize(ds, output_fig=True):
    d = datetime.strptime(ds, '%Y-%m-%d')
    sample_date = d.strftime("%#m/%#d/%y")
    
    df = covid_19_confirmed_China.loc[:, ['Province/State', sample_date]]
    df['province_id'] = [sheng_name_to_id[w[1]['Province/State']] for w in df.iterrows()]
    
    map_with_covid_19 = map_cn.set_index('SHENG').join(df.set_index('province_id'))
    
    vmin = 1
    vmax = 100000
    
    fig, ax = plt.subplots(1, figsize=(20,15))
    ax.axis('off')
    ax.set_title(ds)
    
    map_with_covid_19.plot(column=sample_date, cmap='Reds', linewidth=0.2, edgecolor='black', figsize=(30,30), legend=True, ax=ax, norm=plt.Normalize(vmin=vmin, vmax=vmax), vmin=vmin, vmax=vmax)
    
    if output_fig:
        plt.savefig('c:/tmp/fig/China_{0}.png'.format(ds))

In [ ]:
plot_normalize('2020-2-22', False)

In [ ]:
from datetime import datetime
import matplotlib.colors as colors

def plot_lognorm(ds, output_fig=True):
    d = datetime.strptime(ds, '%Y-%m-%d')
    sample_date = d.strftime("%#m/%#d/%y")
    
    df = covid_19_confirmed_China.loc[:, ['Province/State', sample_date]]
    df['province_id'] = [sheng_name_to_id[w[1]['Province/State']] for w in df.iterrows()]
    
    map_with_covid_19 = map_cn.set_index('SHENG').join(df.set_index('province_id'))
    
    vmin = 1
    vmax = 100000
    
    fig, ax = plt.subplots(1, figsize=(20,15))
    ax.axis('off')
    ax.set_title(ds)
    
    map_with_covid_19.plot(column=sample_date, cmap='Reds', linewidth=0.2, edgecolor='black', figsize=(30,30), legend=True, ax=ax, norm=colors.LogNorm(vmin=vmin, vmax=vmax), vmin=vmin, vmax=vmax)

    if output_fig:
        plt.savefig('c:/tmp/fig/China_{0}.png'.format(ds))

In [ ]:
plot_lognorm('2020-2-22', False)

In [ ]:
from datetime import datetime
import matplotlib.colors as colors
import matplotlib.cm as cm
import numpy as np

def plot_map_colorbar(ds, output_fig=True):
    d = datetime.strptime(ds, '%Y-%m-%d')
    sample_date = d.strftime("%#m/%#d/%y")
    
    df = covid_19_confirmed_China.loc[:, ['Province/State', sample_date]]
    df['province_id'] = [sheng_name_to_id[w[1]['Province/State']] for w in df.iterrows()]
    
    map_with_covid_19 = map_cn.set_index('SHENG').join(df.set_index('province_id'))
    
    vmin = 0
    vmax = 1e+5
    
    fig, ax = plt.subplots(1, figsize=(20,15))
    ax.axis('off')
    ax.set_title(ds)
    
    bounds = np.array([0, 1, 1e+1, 50, 1e+2, 1e+3, 1e+4, 1e+5])
    norm = colors.BoundaryNorm(boundaries=bounds, ncolors=256)
    
    map_with_covid_19.plot(column=sample_date, cmap='Reds', linewidth=0.2, edgecolor='black', ax=ax, norm=norm, vmin=vmin, vmax=vmax)

    cb = fig.colorbar(cm.ScalarMappable(norm=norm, cmap='Reds'), ax=ax)
    cb.set_ticklabels(['0', '1', '10-49', '50-99', '100-999', '10000-99999', '>=100000'])
    
    if output_fig:
        plt.savefig('c:/tmp/fig/China_{0}.png'.format(ds))

In [ ]:
plot_map_colorbar('2020-1-22', False)

In [ ]:
from datetime import datetime
import matplotlib.colors as colors
import matplotlib.cm as cm
import matplotlib.patches as mpatches
import numpy as np

import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'sans-serif'  
plt.rcParams['font.sans-serif'] = 'SimSun,Times New Roman'# 中文设置成宋体，除此之外的字体设置成New Roman  

def plot_map_legend(ds, output_fig=True):
    d = datetime.strptime(ds, '%Y-%m-%d')
    sample_date = d.strftime("%#m/%#d/%y")
    
    df = covid_19_confirmed_China.loc[:, ['Province/State', sample_date]]
    df['province_id'] = [sheng_name_to_id[w[1]['Province/State']] for w in df.iterrows()]
    
    map_with_covid_19 = map_cn.set_index('SHENG').join(df.set_index('province_id'))
    
    vmin = 0
    vmax = 1e+5
    
    fig, ax = plt.subplots(1, figsize=(15,15))
    ax.axis('off')
    ax.set_title(ds)
    
    bounds = np.array([0, 1, 1e+1, 50, 1e+2, 1e+3, 1e+4, 1e+5])
    norm = colors.BoundaryNorm(boundaries=bounds, ncolors=256)
    
    map_with_covid_19.plot(column=sample_date, cmap='Reds', linewidth=0.2, edgecolor='black', ax=ax, norm=norm, vmin=vmin, vmax=vmax)

    rgba = cm.ScalarMappable(norm=norm, cmap='Reds').to_rgba(bounds)
    handles = [mpatches.Patch(color=c) for c in rgba]
    
    labels = ['0', '1-9', '10-49', '50-99', '100-999', '1000-9999', '≥10000']
    leg = fig.legend(handles=handles, labels=labels, fontsize=14, title='图例（单位：人）', title_fontsize=14, loc=(0.12,0.24))
    
    if output_fig:
        plt.savefig('c:/tmp/fig/China_{0}.png'.format(ds))

In [ ]:
plot_map_legend('2020-1-22', True)

In [ ]:
plot_map_legend('2020-1-22', True)

In [ ]:
plot_map_legend('2020-2-22', True)

In [ ]:
plot_map_legend('2020-3-22', True)

In [ ]:
plot_map_legend('2020-4-22', True)

In [ ]:
plot_map_legend('2020-5-22', True)

In [ ]:
plot_map_legend('2020-6-22', True)

In [ ]:
plot_map_legend('2020-7-22', True)

In [ ]:
plot_map_legend('2020-8-22', True)

In [ ]:
plot_map_legend('2020-9-22', True)